In [0]:
!pip install -U rapidfuzz
!pip install -U gensim
%pip install -U pip

Python interpreter will be restarted.
Requirement already satisfied: pip in /local_disk0/.ephemeral_nfs/envs/pythonEnv-050f48af-17f7-4f05-b918-34bf52ef9702/lib/python3.8/site-packages (21.0.1)
Collecting pip
 Using cached pip-23.2.1-py3-none-any.whl (2.1 MB)
Installing collected packages: pip
 Attempting uninstall: pip
 Found existing installation: pip 21.0.1
 Uninstalling pip-21.0.1:
 Successfully uninstalled pip-21.0.1
Successfully installed pip-23.2.1
Python interpreter will be restarted.

In [0]:
import pandas as pd, numpy as np
DBFS = "/dbfs/FileStore/tables/OFFSHORE/"
DBFO = "/dbfs/FileStore/tables/OFFSHORE/"
DBFM = "/dbfs/FileStore/tables/MALLIK/"
DBFR = "/dbfs/FileStore/tables/OFFSHORE_RESULTS/"

pd.options.display.max_columns = None
pd.options.display.max_rows = 30

In [0]:
import pandas as pd, numpy as np
from rapidfuzz import process, fuzz
np.set_printoptions(threshold=np.inf)

In [0]:
ic_gtins = pd.read_csv(DBFO + 'ice_cream_items.csv')
ic_gtins['GTIN_NO'] = ic_gtins.GTIN_NO.apply(lambda x: int(x)).astype(str).apply(lambda x: (14- len(x))*'0'+x)

In [0]:
# ic_gtins_KFT_cols = pd.read_csv('icecream_additional_ITEM_VIEW.csv')
results = pd.read_csv(DBFO + 'ICECREAM_FLAVORS_101023_v2.csv')

ic_additional_VND_ECOM_DSC = pd.read_csv(DBFO + 'icecream_additional_VND_ECOM_DSC.csv')
ic_gtins_KFT_cols = ic_gtins_KFT_cols.merge(ic_additional_VND_ECOM_DSC, how = 'left', on = 'ITM_ID')
ic_gtins_KFT_cols['GTIN_NO'] = ic_gtins_KFT_cols.GTIN_NO.astype(str).apply(lambda x: (14-len(x))*'0'+x)

In [0]:
ic_gtins.replace('(null)', None, inplace= True)
ic_gtins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12386 entries, 0 to 12385
Data columns (total 17 columns):
 # Column Non-Null Count Dtype 
--- ------ -------------- ----- 
 0 GTIN_NO 12386 non-null object
 1 ECOMMERCE_DESCRIPTION 12386 non-null object
 2 RECEIPT_DESCRIPTION 9402 non-null object
 3 TAG_DESCRIPTION 11684 non-null object
 4 PRODUCT_DESCRIPTION/MKT_MSG 11371 non-null object
 5 KROGER_OWNED_FLAVOR 153 non-null object
 6 FLV_DSC 1322 non-null object
 7 PID_COM_DSC 9194 non-null object
 8 PID_SUBCOM_DSC 9194 non-null object
 9 CFIC_DEPT_NAM 4277 non-null object
 10 CFIC_COM_NAM 4277 non-null object
 11 CFIC_SUBCOM_NAM 4277 non-null object
 12 Unnamed: 12 13 non-null object
 13 Unnamed: 13 7 non-null object
 14 Unnamed: 14 1 non-null object
 15 Unnamed: 15 1 non-null object
 16 Unnamed: 16 1 non-null object
dtypes: object(17)
memory usage: 1.6+ MB

In [0]:
ic_gtins_KFT_cols.DPT_DSC.unique()

array(['FROZEN GROCERY', 'BAKERY', 'NATURAL FOODS', 'GROC-ALL OTHER',
       'FRESH FOOD DEST', 'COMMERCIAL BKY', 'HOUSEWARES',
       'PACKAGE PRODUCE', 'POULTRY', 'REFRIG GROCERY', 'HEALTH', 'DAIRY',
       'COFFEE SHOP', 'WINE', 'CANDY', 'CKY/CRKR/SNK'], dtype=object)

In [0]:
ic_gtins['GTIN_NO'] = ic_gtins.GTIN_NO.apply(lambda x: int(x)).astype(str).apply(lambda x: (14- len(x))*'0'+x)

In [0]:
# Flavor List and desired mappings From business (interim) based on "Initial ice cream flavor list.xlsx"
IC_flavors_new =\
['VANILLA','CHOCOLATE','FRUIT','COOKIES AND CREAM','CHOCOLATE CHIP','CARAMEL','TOFFEE','MINT CHOCOLATE CHIP','PEANUT BUTTER','COOKIE DOUGH',\
'BUTTER PECAN','COFFEE','STRAWBERRY','MOOSE TRACKS','ROCKY ROAD','COTTON CANDY','BIRTHDAY CAKE','PUMPKIN','RED VELVET','NEOPOLITAN']
flavor_maps = {'MOCHA':'COFFEE','BROWNIE':'CHOCOLATE','COCOA':'CHOCOLATE','TOFFEE':'CARAMEL','CHERRY':'FRUIT', 'LEMON': 'FRUIT',
 'ORANGE': 'FRUIT',
 'LIME': 'FRUIT',
 'PEACH': 'FRUIT',
 'BANANA': 'FRUIT',
 'PINEAPPLE': 'FRUIT',
 'BERRY': 'FRUIT',
 'APPLE': 'FRUIT',
 'MANGO': 'FRUIT',
 'RASPBERRY': 'FRUIT',
 'PASSION FRUIT': 'FRUIT',
 'POMEGRANATE': 'FRUIT',
 'BLUEBERRY': 'FRUIT'}
IC_flavors_new += flavor_maps.keys()

In [0]:
### Remove substrings like 'CHOCOLATE' if 'CHOCOLATE CHIP' is also present as a top result
# string_list = [bingo[0:2] for bingo in matched]

def remove_substrings(string_list):
    str_df = pd.DataFrame(string_list, columns=['Sentence', 'Scores'])
    str_df['Word Length'] = str_df.Sentence.apply(lambda x: len(x.split(' ')))
    str_df.sort_values('Word Length', inplace= True)
    str_df.reset_index(drop = True, inplace= True)
    str_dff = str_df[str_df.Scores==100]
    if len(str_dff.index)==0:
        flag = 1
        return str_df.Sentence.tolist()
    if len(str_dff.index)==1:
        return str_dff.Sentence.tolist()
    str_dff['Drop'] = ''
    for iterr in range(len(str_dff)):
        str_1 = str_dff.Sentence.iloc[iterr]
        if iterr==len(str_dff):
            break
        for sent in str_dff.Sentence.iloc[iterr+1:]:
            if str_1 in sent:
                str_dff.Drop.iloc[iterr] = 'Yes'
                break
            if len(str_df) < 2:
                break
    return str_dff[str_dff.Drop != 'Yes'].Sentence.tolist()

In [0]:
process.extract('STRAWBERRYICE', IC_flavors_new, scorer = fuzz.token_set_ratio)

Out[37]: [('STRAWBERRY', 86.95652173913044, 12),
 ('RASPBERRY', 63.63636363636363, 34),
 ('BERRY', 55.55555555555556, 31),
 ('BLUEBERRY', 45.45454545454545, 37),
 ('CHERRY', 42.10526315789474, 24)]

In [0]:
# Two step matching - 'token set ratio' with threshold, followed by 'ratio'
def flavor_match(item, flavor_list, cutoff = 90):
    matched = process.extract(item, flavor_list, score_cutoff= cutoff, scorer = fuzz.token_set_ratio)
    flavor_shortlist = remove_substrings([i[0] for i in matched])
    matched = [i for i in matched if i[0] in flavor_shortlist]
    if len( flavor_shortlist) >0:
        max_score = matched[0][1]
        final_match = [ i for i in matched if i[1]== max_score]
        match_list = [i[0] for i in final_match]
        match_list = [flavor_maps[i] if i in flavor_maps.keys() else i for i in match_list]
        # if set(match_list) == {'STAWBERRY','FRUIT'}: 
        #     match_list = ['FRUIT'] # Drop Strawberry if fruit is the only other flavor
        if  set(match_list) != {'FRUIT'} : match_list = [ i for i in match_list if i != 'FRUIT'] # Drop Fruit if it appears with something else
        matched = ','.join(list(pd.Series(match_list).unique()))     
    else: matched = 'OTHER'
    return matched

In [0]:
ic_gtins_KFT_cols['Assigned Flavor(s)']= \
ic_gtins_KFT_cols.VND_ECOM_DSC.str.upper().apply(lambda x:flavor_match(x, flavor_list= IC_flavors_new))
ic_gtins_KFT_cols['Flavor_Count'] = ic_gtins_KFT_cols['Assigned Flavor(s)'].apply(lambda x: len(x.split(',')))
#Convert GTIN to 14 digit format
ic_gtins = ic_gtins.merge(ic_gtins_KFT_cols, how = 'left', on = 'GTIN_NO')

In [0]:
ic_gtins = ic_gtins[ic_gtins.CFIC_SUBCOM_NAM != 'Toppings']# Remove "Toppings"

In [0]:
ic_gtins['Assigned Flavor(s)'].value_counts()

OTHER                                      5606
CHOCOLATE                                  1285
FRUIT                                      1189
VANILLA                                    1139
CARAMEL                                     583
                                           ... 
VANILLA,CHOCOLATE,CARAMEL,PEANUT BUTTER       1
CHOCOLATE,RED VELVET                          1
VANILLA,BUTTER PECAN                          1
CHOCOLATE,BUTTER PECAN                        1
COFFEE,CHOCOLATE                              1
Name: Assigned Flavor(s), Length: 64, dtype: int64

In [0]:
ic_gtins[['GTIN_NO','VND_ECOM_DSC','Assigned Flavor(s)']].to_csv('ICECREAM_FLAVORS_101023_v2.csv', index = None)

# UPDATED 10/10/23

In [0]:
other_results = results[results['Assigned Flavor(s)'] == 'OTHER']
other_results = other_results[~other_results.VND_ECOM_DSC.isna()]
print(other_results.shape) # 5605

other_results['VND_ECOM_DSC_altered'] = ''
other_results.VND_ECOM_DSC_altered = other_results.VND_ECOM_DSC.apply(lambda x: x.upper().replace('STRAWBERRY', ' STRAWBERRY ').replace('STRAWBERRIES', 'STRAWBERRY'))
other_results[other_results.VND_ECOM_DSC_altered.str.contains('STRAWBER')].shape

In [0]:
matched = []
for item in other_results[other_results.VND_ECOM_DSC_altered.str.contains('STRAWBER')]['VND_ECOM_DSC_altered'].tolist():
    matched.append([i[:2] for i in process.extract(item, IC_flavors_new, scorer = fuzz.token_set_ratio)])

straw = other_results[other_results.VND_ECOM_DSC_altered.str.contains('STRAWBER')]
straw['Flavor 1'] = [matched[i][0][0] for i in range(len(matched))]
straw['Scores 1'] = [matched[i][0][1] for i in range(len(matched))]
straw['Flavor 2'] = [matched[i][1][0] for i in range(len(matched))]
straw['Scores 2'] = [matched[i][1][1] for i in range(len(matched))]
straw['Flavor 3'] = [matched[i][2][0] for i in range(len(matched))]
straw['Scores 3'] = [matched[i][2][1] for i in range(len(matched))]
straw['Flavor 4'] = [matched[i][3][0] for i in range(len(matched))]
straw['Scores 4'] = [matched[i][3][1] for i in range(len(matched))]
straw['Flavor 5'] = [matched[i][4][0] for i in range(len(matched))]
straw['Scores 5'] = [matched[i][4][1] for i in range(len(matched))]

straw["Flavor 100%"] = ''
for i in range(len(straw)):
    print(i)
    final = ''
    if straw['Scores 1'].iloc[i] == 100:
        final+=straw['Flavor 1'].iloc[i]
    if straw['Scores 2'].iloc[i] == 100:
        final+=', '
        final+=straw['Flavor 2'].iloc[i]
    if straw['Scores 3'].iloc[i] == 100:
        final+=', '
        final+=straw['Flavor 3'].iloc[i]
    if straw['Scores 4'].iloc[i] == 100:
        final+=', '
        final+=straw['Flavor 4'].iloc[i]
    if straw['Scores 5'].iloc[i] == 100:
        final+=', '
        final+=straw['Flavor 5'].iloc[i]
    straw.at[i, "Flavor 100%"] = final


straw['Assigned Flavor(s)'] = 'OTHER'
straw = straw[['GTIN_NO', 'VND_ECOM_DSC', 'Assigned Flavor(s)', 'Flavor 100%', 'Flavor 1', 'Scores 1', 'Flavor 2', 'Scores 2', 'Flavor 3', 'Scores 3',
       'Flavor 4', 'Scores 4', 'Flavor 5', 'Scores 5']]
straw.to_csv(DBFO + 'STRAWBERRY.csv', index=False)

In [0]:
from statistics import mode
matrix=[other_results.VND_ECOM_DSC.iloc[i].replace('-', ' ').split(' ') for i in range(len(other_results.VND_ECOM_DSC))]
pd.Series([item for row in matrix for item in row]).value_counts().reset_index().head()

Out[309]:

,index,0
0,,4140
1,Cream,1523
2,Ice,1439
3,Salt,814
4,ICE,595


In [0]:
rest = other_results[~other_results.GTIN.isin([straw.GTIN.tolist()])]

matched = []
for item in rest['VND_ECOM_DSC'].tolist():
    matched.append([i[:2] for i in process.extract(item, IC_flavors_new, scorer = fuzz.token_set_ratio)])

rest['Flavor 1'] = [matched[i][0][0] for i in range(len(matched))]
rest['Scores 1'] = [matched[i][0][1] for i in range(len(matched))]
rest['Flavor 2'] = [matched[i][1][0] for i in range(len(matched))]
rest['Scores 2'] = [matched[i][1][1] for i in range(len(matched))]
rest['Flavor 3'] = [matched[i][2][0] for i in range(len(matched))]
rest['Scores 3'] = [matched[i][2][1] for i in range(len(matched))]
rest['Flavor 4'] = [matched[i][3][0] for i in range(len(matched))]
rest['Scores 4'] = [matched[i][3][1] for i in range(len(matched))]
rest['Flavor 5'] = [matched[i][4][0] for i in range(len(matched))]
rest['Scores 5'] = [matched[i][4][1] for i in range(len(matched))]

rest.to_csv(DBFO + 'toMallik.csv')